# Creating LDA Topic Model with Gensim
## Introduction

LDA’s approach to topic modeling is to classify text in a document to a particular topic. Modeled as Dirichlet distributions, LDA builds a topic per document model and words per topic model. After providing the LDA topic model algorithm, in order to obtain a good composition of topic-keyword distribution, it re-arrange the topics distributions within the document and the keywords distribution within the topics. Here, we are going to use LDA (Latent Dirichlet Allocation) to extract the naturally discussed topics from dataset. 

## Loading Data Set

The dataset which we are going to use is the dataset of ’20 Newsgroups’ having thousands of news articles from various sections of a news report. It is available under Sklearn data sets. We used the following Python script to download it and we can also look at some of the sample news.

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
data = newsgroups_train.data
newsgroups_train.data[:4]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

## Prerequisite

We need Stopwords from NLTK and English model from Scapy. Both can be downloaded as follows:

In [2]:
import nltk;
import spacy;
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fpere\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importing Necessary Packages

In order to build LDA model we need to import following necessary package:

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

## Preparing Stopwords 

Now, we need to import the Stopwords and use them:

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Clean up the Text

Now, with the help of Gensim’s simple_preprocess() we need to tokenise each sentence into a list of words. We should also remove the punctuations and unnecessary characters. In order to do this, we will create a function named sent_to_words():

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

## Building Bigram & Trigram Models

Bigrams are two words that are frequently occurring together in the document and trigram are three words that are frequently occurring together in the document. With the help of Gensim’s Phrases model, we can do this:

In [6]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

## Filter out Stopwords

Next, we need to filter out the Stopwords. Along with that, we will also create functions to make bigrams, trigrams and for lemmatisation:

In [7]:
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
   return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
     doc = nlp(" ".join(sent))
     texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

## Building Dictionary & Corpus for Topic Model

We now need to build the dictionary & corpus:

In [8]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[
   'NOUN', 'ADJ', 'VERB', 'ADV'
])
print(data_lemmatized[:4]) #it will print the lemmatized data.

id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

[['lerxst', 'thing', 'car', 'nntp_posting', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['final', 'call', 'summary', 'final', 'call', 'clock', 'report', 'keyword', 'acceleration', 'line', 'host', 'fair', 'number', 'brave', 'soul', 'upgrade', 'clock', 'oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'top', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'hour', 'usage', 'day', 'functionality', 'floppy', 'especially', 'request', 'summarize', 'next', 'day', 'add', 'network', 'knowledge', 'base', 'do', 'clock', 'answer', 'poll', 'thank', 'guy_kuo'], ['computer', 'network', 'di

## Building LDA Topic Model

We already implemented everything that is required to train the LDA model. Now, it is the time to build the LDA topic model. For our implementation example, it can be done with the help of following line of codes:

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(
   corpus=corpus, id2word=id2word, num_topics=20, random_state=100, 
   update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True
)

## Viewing Topics in LDA Model

The LDA model (lda_model) we have created above can be used to view the topics from the documents. It can be done with the help of following script:

In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.088*"israeli" + 0.054*"route" + 0.052*"library" + 0.049*"bomb" + '
  '0.036*"link" + 0.035*"authority" + 0.032*"arab" + 0.027*"creation" + '
  '0.025*"trap" + 0.023*"regardless"'),
 (1,
  '0.033*"system" + 0.024*"use" + 0.020*"thank" + 0.019*"program" + '
  '0.017*"mail" + 0.017*"include" + 0.016*"information" + 0.016*"send" + '
  '0.015*"line" + 0.015*"also"'),
 (2,
  '0.018*"car" + 0.016*"high" + 0.015*"new" + 0.013*"buy" + 0.013*"year" + '
  '0.012*"power" + 0.011*"price" + 0.011*"sell" + 0.011*"large" + '
  '0.010*"cost"'),
 (3,
  '0.098*"child" + 0.091*"kill" + 0.053*"woman" + 0.046*"fire" + 0.046*"death" '
  '+ 0.042*"soldier" + 0.042*"man" + 0.035*"shoot" + 0.033*"murder" + '
  '0.033*"attack"'),
 (4,
  '0.182*"atheist" + 0.108*"wing" + 0.076*"atheism" + 0.015*"pure" + '
  '0.009*"assault" + 0.008*"swing" + 0.007*"silver" + 0.004*"crusade" + '
  '0.004*"bobby_mozumder" + 0.002*"ideological"'),
 (5,
  '0.082*"faith" + 0.076*"book" + 0.061*"religion" + 0.060*"evidence" +

## Computing Model Perplexity

The LDA model (lda_model) we have created above can be used to compute the model’s perplexity, i.e. how good the model is. The lower the score the better the model will be. It can be done with the help of following script:

In [11]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -13.063662303127526


## Visualising the Topics-Keywords

The LDA model (lda_model) we have created above can be used to examine the produced topics and the associated keywords. It can be visualised by using pyLDAvis package as follows:

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.357374  0.105276       1        1  39.370548
13    -0.323488  0.203986       2        1  19.597479
1     -0.308906 -0.098243       3        1  12.149929
2     -0.280228 -0.125907       4        1  10.871031
9     -0.128998 -0.286661       5        1   3.265305
6     -0.025292  0.176377       6        1   2.388477
12    -0.056526 -0.092728       7        1   1.864691
16     0.046241  0.012412       8        1   1.462641
19     0.116143 -0.010604       9        1   1.286917
3      0.054324  0.097401      10        1   1.257772
5      0.039700  0.082475      11        1   1.245433
8      0.101631 -0.023562      12        1   0.977386
7      0.105791 -0.025121      13        1   0.924958
18     0.111787 -0.011437      14        1   0.819106
17     0.129330  0.030899      15        1   0.554792
0      0.142523 -0.004580      16        1   0.498247
15     0.149079 -0.007742      17        1   0.492405
10     0.153681 -0.007453      18        1   0.422678
14     0.154565 -0.009419      19        1   0.388151
4      0.176016 -0.005368      20        1   0.162060, topic_info=              Term          Freq         Total Category  logprob  loglift
3760            ax   2974.000000   2974.000000  Default  30.0000  30.0000
196          write  13415.000000  13415.000000  Default  29.0000  29.0000
39            year   4625.000000   4625.000000  Default  28.0000  28.0000
242        believe   3162.000000   3162.000000  Default  27.0000  27.0000
1797           max   2487.000000   2487.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
3905        theory      0.025831      1.204035  Topic20 -11.1672   2.5831
4291        hockey      0.025832      1.204079  Topic20 -11.1672   2.5831
5126       playoff      0.025831      1.204053  Topic20 -11.1672   2.5831
3397  christianity      0.025831      1.204031  Topic20 -11.1673   2.5831
4848      american      0.025831      1.204056  Topic20 -11.1673   2.5831

[821 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
475      18  0.993787        absolute
2334     18  0.562933  absolute_truth
1226     17  0.998460          accept
865       2  0.732502        accident
865      14  0.059246        accident
...     ...       ...             ...
452       1  0.998922           wrong
39        1  0.230039            year
39        2  0.199771            year
39        4  0.331438            year
39        7  0.238255            year

[981 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 14, 2, 3, 10, 7, 13, 17, 20, 4, 6, 9, 8, 19, 18, 1, 16, 11, 15, 5])